In [0]:
#Import necessary libraries for operations

import pandas as pd  #pandas library to read and manipulate data 
import requests      #requests library to perform request from spotify api
import datetime      #datetime library for date/time operation


In [0]:

TOKEN = "BQBfHQizGXPLpPomiPAoZefDLyVCOa_fspxPZu7hlCa-yktiiIJwhz_tOCWcrV-MbAlC4n8auWcAGPxA9fY0ci5Fq_uliUjFLCltW_dTcyq9t97rx3ezHyKlWOYc-QlQHIC-G-khXmLnVlA0QPbVWjiw-k5A5G2ztA65FbPzEQlL5ar3q26AayZhNs8QWFAnU3Lki4tI"    

# Spotify API token
# Generate your token here:  https://developer.spotify.com/console/get-recently-played/
# Note: You need a Spotify account (can be easily created for free). Token expires after 1 hour

In [0]:
#Extract data


#Define GET url headers 
headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=TOKEN)
    }
    

#Convert time to Unix timestamp in miliseconds      
today = datetime.datetime.now()          #instantiate the current datetime
yesterday = today - datetime.timedelta(days=1)       #instantiate "yesterday's" datetime
yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000   #convert timestamp to unixtimestamp as requested by spotifiy API

#Download all songs you've listened to "after yesterday", which means in the last 24 hours      
data = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers = headers).json()   #.json() method converts returned data to the 'readable' json format 

In [0]:
#Create empty lists to hold song information
song_names = []
artist_names = []
played_at_list = []
timestamps = []


# Loops through each song item returned to extract only the relevant bits of data from the json object      
for song in data["items"]:     
    song_names.append(song["track"]["name"])     #song name
    artist_names.append(song["track"]["album"]["artists"][0]["name"])    #artist name
    played_at_list.append(song["played_at"])   #timestamp song was played; serves as PK of table
    timestamps.append(song["played_at"][0:10])    #date song was played extracted from time stamp
        
# Prepare a dictionary in order to turn it into a pandas dataframe       
song_dict = {
    "song_name" : song_names,
    "artist_name": artist_names,
    "played_at" : played_at_list,
    "timestamp" : timestamps
}

songs_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])
print(songs_df)  #check to see the data returned



                            song_name    artist_name  \
0                     Head Over Heels  Show Dem Camp   
1                           Hood2Hood            Nas   
2                           Hood2Hood            Nas   
3                                  30            Nas   
4                    Michael & Quincy            Nas   
5                               Legit            Nas   
6                     Ghetto Reporter            Nas   
7                          I Need You         Vector   
8                 Teslim Introduction         Vector   
9                               Intro  Show Dem Camp   
10                      Plenty Loving         Wizkid   
11                           Pressure         Wizkid   
12  Wow (feat. Skepta & Naira Marley)         Wizkid   
13                        Flower Pads         Wizkid   
14                               Deep         Wizkid   

                   played_at   timestamp  
0   2022-11-13T16:14:23.209Z  2022-11-13  
1   2022-11-13T16

In [0]:
#Transformation/Validation step
def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Check if dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing execution")
        return False 

    # Primary Key Check
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("Primary Key check is violated")

    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null values found")

    return True


#Validate
if check_if_valid_data(songs_df):
        print("Data valid, proceed to Load stage")


Data valid, proceed to Load stage


In [0]:
# Load
#convert pandas dataframe to spark dataframe
spark_df = spark.createDataFrame(songs_df)  

#saving to table
spark_df.write.mode("append").saveAsTable("load_spotify_data")    #table is autocreated if it doesn't already exist, if it exists, new data is appended to it.

#Validate write operation
print("Loaded successfully")  

Loaded successfully


In [0]:
#Query table to confirm data load
df=sqlContext.sql("SELECT * FROM load_spotify_data ORDER BY played_at DESC")
df.show()

+--------------------+-------------+--------------------+----------+
|           song_name|  artist_name|           played_at| timestamp|
+--------------------+-------------+--------------------+----------+
|     Head Over Heels|Show Dem Camp|2022-11-13T16:14:...|2022-11-13|
|           Hood2Hood|          Nas|2022-11-13T16:13:...|2022-11-13|
|           Hood2Hood|          Nas|2022-11-13T15:50:...|2022-11-13|
|                  30|          Nas|2022-11-13T15:39:...|2022-11-13|
|    Michael & Quincy|          Nas|2022-11-13T15:37:...|2022-11-13|
|               Legit|          Nas|2022-11-13T15:30:...|2022-11-13|
|     Ghetto Reporter|          Nas|2022-11-13T15:26:...|2022-11-13|
|          I Need You|       Vector|2022-11-13T14:55:...|2022-11-13|
| Teslim Introduction|       Vector|2022-11-13T14:29:...|2022-11-13|
|               Intro|Show Dem Camp|2022-11-13T08:59:...|2022-11-13|
|       Plenty Loving|       Wizkid|2022-11-12T18:33:...|2022-11-12|
|            Pressure|       Wizki